In [1]:
%gui qt


In [8]:
import napari



ModuleNotFoundError: No module named 'napari'

We can then import `napari`, our `nbscreenshot` utility and instantiate an empty viewer.

In [2]:
import napari
from napari.utils import nbscreenshot

# Create an empty viewer
viewer = napari.Viewer()

ModuleNotFoundError: No module named 'napari'

In [43]:
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_3d,edge_preserving_smoothing_3d
from skimage.morphology import remove_small_objects, binary_closing, ball, disk, erosion, binary_dilation, binary_erosion   # function for post-processing (size filter)
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import get_middle_frame, get_3dseed_from_mid_frame
from skimage import transform, measure
import h5py
from tqdm import tqdm
from pandas import DataFrame
import warnings
import h5py
from skimage.filters import sobel, threshold_otsu, scharr, gaussian
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.morphology import binary_closing, ball,disk, binary_opening, local_maxima
from skimage.measure import label, regionprops
from scipy.ndimage import distance_transform_edt
from aicssegmentation.core.utils import hole_filling
import trackpy as tp
import scipy.ndimage as ndi
import tkinter as tk
from tkinter import filedialog
from skimage.restoration import rolling_ball
from skimage.restoration import ellipsoid_kernel

In [4]:
# Use tkinter to interactively select files to import
root = tk.Tk()
root.withdraw()

my_filetypes = [('all files', '.*'),('Image files', '.hdf5')]

filename = filedialog.askopenfilenames(parent = root, title='Please Select a File', filetypes = my_filetypes)[0]

f = h5py.File(filename, 'r')

lmb2_image = f['488 Channel'][:]
#cpla2_intensity_image = f['561 Channel'][:]


In [45]:
test_img =lmb2_image[25]

In [240]:
test1_img = lmb2_image[25]

In [46]:
viewer.add_image(test_img)

<Image layer 'test_img' at 0x794bee48>

In [47]:
smooth_img = edge_preserving_smoothing_3d(test_img, numberOfIterations=4)

In [26]:
smooth_img = image_smoothing_gaussian_3d(test_img, sigma=1)

In [208]:
smooth_img_1 = image_smoothing_gaussian_3d(test_img, sigma=1)


In [28]:
viewer.add_image(smooth_img)

<Image layer 'smooth_img [1]' at 0x7185a648>

In [48]:
background = rolling_ball(
    smooth_img,
    kernel=ellipsoid_kernel(
        (1, 20, 20),
        0.2
    )
)


In [95]:
viewer.add_image(background)

<Image layer 'background' at 0xf534ec88>

In [49]:
viewer.add_image(smooth_img-background)

<Image layer 'Image' at 0x794c6908>

In [50]:
new_img = smooth_img - background

In [57]:
bw, object_for_debug = MO(new_img, global_thresh_method='ave', extra_criteria=True, object_minArea=4000, return_object=True)

bw = binary_closing(bw,selem=np.ones((3,3,3)))
bw = hole_filling(bw, 1, 40000, fill_2d=True)
bw = remove_small_objects(bw>0, min_size=6000, connectivity=1, in_place=False)
bw = clear_border(bw, mask = mask)


In [115]:
new_img.shape

(26, 512, 512)

In [58]:
viewer.add_image(bw)

<Image layer 'bw [1]' at 0xe293b208>

In [157]:
a = get_middle_frame_area(bw)
mid_slice = bw[a]
viewer.add_image(mid_slice)

<Image layer 'mid_slice' at 0x7e85c088>

In [158]:
distance = distance_transform_edt(mid_slice)
viewer.add_image(distance)


<Image layer 'distance' at 0x1002f1508>

In [161]:
smoothed_distance = gaussian(distance, 3)
viewer.add_image(smoothed_distance)

<Image layer 'smoothed_distance' at 0x108530f48>

In [173]:
label(mid_slice)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [177]:
peak_local_max = peak_local_max(
    smoothed_distance,
    footprint=np.ones((42, 42), dtype=np.bool),
)
peaks = np.nonzero(peak_local_max)

TypeError: 'numpy.ndarray' object is not callable

In [163]:
peaks

(array([ 66, 141, 142, 224, 313, 369, 405], dtype=int64),
 array([387, 259, 257, 225, 442, 434, 156], dtype=int64))

In [164]:
np.array(peaks).T

array([[ 66, 387],
       [141, 259],
       [142, 257],
       [224, 225],
       [313, 442],
       [369, 434],
       [405, 156]], dtype=int64)

In [165]:

viewer.add_points(np.array(peaks).T, name='peaks', size=5, face_color='red');

In [20]:
#Make a mask to clear border
from skimage.segmentation import clear_border
mask = np.ones(new_img.shape)

for n in range(mask.shape[0]):
   im_slice = mask[n,:,:]
   im_slice[0,:] = 0
   im_slice[:,0] = 0
   im_slice[:,-1] = 0
   im_slice[-1,:] = 0


mask = mask.astype(np.bool)

In [78]:
mask.shape

(26, 512, 512)

In [156]:
# Make a function to get middle frame based on segmented area
def get_middle_frame_area(labelled_image_stack):
    max_area = 0
    max_n = 0
    for z in range(labelled_image_stack.shape[0]):
       img_slice = labelled_image_stack[z,:,:]
       area = np.count_nonzero(img_slice)
       
       if area >= max_area:
          max_area = area
          max_n = z
    
    return max_n

In [8]:
def get_3dseed_from_mid_frame(bw, stack_shape, mid_frame, hole_min, bg_seed = True):
    from skimage.morphology import remove_small_objects
    out = remove_small_objects(bw>0, hole_min)

    out1 = label(out)
    stat = regionprops(out1)
    
    # build the seed for watershed
    seed = np.zeros(stack_shape)
    seed_count=0
    if bg_seed:
        seed[0,:,:] = 1
        seed_count += 1

    for idx in range(len(stat)):
        py, px = np.round(stat[idx].centroid)
        seed_count+=1
        seed[mid_frame,int(py),int(px)]=seed_count

    return seed

Let's read the original image from previous lessons, take a maximum projection, and view it in napari:

In [10]:
import scipy.ndimage as ndi
################################
## PARAMETERS for this step ##
intensity_scaling_param = [4000]
gaussian_smoothing_sigma = 3
################################

#struct_img = intensity_normalization(nuclei_mip, scaling_param=intensity_scaling_param)

structure_img_smooth = ndi.median_filter(nuclei_mip, size = 2) 
#smooth_img = ndi.median_filter(nuclei_mip, size = 1.4 )

#nuclei_map = smooth_img - img_bg

In [18]:
viewer.add_image(bw[mid_z])

<Image layer 'Image' at 0xe99ff888>

In [11]:
viewer.add_image(structure_img_smooth)

<Image layer 'structure_img_smooth' at 0xe4bb1748>

In [19]:
a = viewer.add_points(name='peaks', size=15, face_color='red', ndim=3);

In [20]:
b = a.data

In [27]:
b = np.delete(b, 0, axis=1)

In [28]:
b

array([[179.41939344, 208.01019483],
       [245.71161326, 310.50135595]])

In [22]:
c = distance_transform_edt(bw[mid_z])

In [23]:
viewer.add_image(-c)

<Image layer 'Image [1]' at 0xe9d01108>

In [71]:
seg = watershed(-c, markers=markers,mask =bw[mid_z], watershed_line=True )
seg = clear_border(seg)
            
seg = remove_small_objects(seg>0, min_size=4000, connectivity=1, in_place=False)

In [74]:
viewer.add_labels(label(seg))

<Labels layer 'Labels' at 0xf879f708>

In [29]:
markers = np.zeros(bw[mid_z].shape, dtype=np.uint32)
marker_indices = tuple(np.round(b).astype(int).T)
markers[marker_indices] = np.arange(len(b)) + 1
markers_big = dilation(markers, disk(3))


In [228]:
center_list = np.transpose(np.nonzero(maxima))

In [51]:
np.arange(len(b))+1

array([1, 2])

In [272]:
seed = get_3dseed_from_mid_frame(structure_img_smooth.shape, center_list,mid_z,2000, bg_seed=False)           

TypeError: '>' not supported between instances of 'tuple' and 'int'

In [235]:
seed_big = dilation(seed, ball(5))

In [221]:
markers

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [199]:
for n in range(nuclei_mip.shape[0]):
    nuclei_mip[n,:,:] = binary_opening(nuclei_mip[n,:,:],selem=np.ones((1,1)))

In [15]:
from skimage import morphology
bw = nuclei_mip > threshold_otsu(nuclei_mip)

#bw = structure_img_smooth > threshold_otsu(structure_img_smooth)
bw = ndi.median_filter(bw, size = 1) 

for n in range(bw.shape[0]):
    bw[n,:,:] = binary_opening(bw[n,:,:],selem=np.ones((2,2)))
    bw[n,:,:] = hole_filling(bw[n,:,:], 1, 4000)
    bw[n,:,:] = remove_small_objects(bw[n,:,:]>0, min_size=3000, connectivity=1, in_place=False)
   
bw = clear_border(bw, mask = mask)

mid_z = get_middle_frame(bw)

In [273]:
seed = get_3dseed_from_mid_frame(bw[mid_z],bw.shape, mid_z,2000, bg_seed=False)

In [274]:
peaks = np.nonzero(seed)
viewer.add_points(np.array(peaks).T, name='peaks', size=5, face_color='red');

In [210]:
img_Mip = np.max(bw, axis = 0)

In [16]:
viewer.add_image(bw)

<Image layer 'bw [3]' at 0xe91a1748>

In [270]:
viewer.add_image(bw[mid_z])

<Image layer 'Image [1]' at 0xf10025c8>

In [254]:
dist = img_Mip


maxima = local_maxima(dist)
viewer.add_points(
    np.transpose(np.nonzero(maxima)),
    name='bad points',
    size=4  # points have 3D "extent"
)

<Points layer 'bad points' at 0xf72808c8>

In [217]:
np.transpose(np.nonzero(maxima))

array([[180, 206],
       [231, 326],
       [304,   0],
       [446, 153],
       [511, 337]], dtype=int64)

In [136]:
edges = scharr(bw)

In [33]:
print(np.unique(seed))

[0. 6.]


In [97]:
from skimage.feature import peak_local_max
coords = peak_local_max(c, footprint=np.ones((2, 2)), labels=bw[mid_z])
mask = np.zeros(c.shape, dtype=bool)
mask[tuple(coords.T)] = True
mask = label(mask)
mask = dilation(mask, disk(4))

In [98]:
seg = watershed(struct_img, markers=markers,watershed_line=True)>0
seg = clear_border(seg, mask = mask)
            
#seg = remove_small_objects(seg>0, min_size=200, connectivity=1, in_place=False)
#seg = hole_filling(seg, 1, 40000, fill_2d=True)

NameError: name 'struct_img' is not defined

In [108]:
measure.regionprops_table(seg)['label']

array([1, 2])

In [99]:
viewer.add_image(mask)

<Image layer 'mask [1]' at 0xf7b2c148>

In [171]:
viewer.add_labels(label(seg))

<Labels layer 'Labels' at 0x12dcfc0c8>

In [73]:
bw_mid_z = remove_small_objects(binary_closing(bw_mid_z,disk(1)),2000)
bw_fill_mid_z = hole_filling(bw_mid_z, 0, 40000)


In [77]:
viewer.add_image(seg)

<Image layer 'seg' at 0x7f9dbb1ecfd0>

In [65]:
viewer.add_image(bw_fill_mid_z)

<Image layer 'bw_fill_mid_z' at 0x7fe1be2bd950>

In [54]:
import numpy as np
pt = np.nonzero(seed) 


In [55]:
point = np.stack((pt[0],pt[1],pt[2]), axis = -1)

In [56]:
viewer.add_points(point)

<Points layer 'point' at 0x7fc2b05c1150>

In [79]:
a = np.arange(6).reshape((3,2))

In [80]:
a

array([[0, 1],
       [2, 3],
       [4, 5]])

In [82]:
np.max(a, axis = 1)

array([1, 3, 5])